# Preprocess the datasets

In [1]:
import pandas as pd
import json
from utils import autoparse_year
import numpy as np

## Summaries

In [2]:
df_summary = pd.read_csv(
	'../data/raw/MovieSummaries/plot_summaries.txt',
	sep='\t',
	# index_col='wiki_id',
	names=['wiki_id', 'summary'],
	dtype={'summary': pd.StringDtype()}
)
assert df_summary.index.is_unique
assert df_summary.wiki_id.is_unique
df_summary

,wiki_id,summary
0,23890098,"Shlykov, a hard-working taxi driver and Lyosha..."
1,31186339,The nation of Panem consists of a wealthy Capi...
2,20663735,Poovalli Induchoodan is sentenced for six yea...
3,2231378,"The Lemon Drop Kid , a New York City swindler,..."
4,595909,Seventh-day Adventist Church pastor Michael Ch...
...,...,...
42298,34808485,"The story is about Reema , a young Muslim scho..."
42299,1096473,"In 1928 Hollywood, director Leo Andreyev look..."
42300,35102018,American Luthier focuses on Randy Parsons’ tra...
42301,8628195,"Abdur Rehman Khan , a middle-aged dry fruit se..."


In [3]:
df_summary.to_pickle('../data/generated/preprocessed/summary.pkl')

### Movies dataset

In [4]:
df_movies = pd.read_csv(
	'../data/raw/MovieSummaries/movie.metadata.tsv', sep='\t',
	# index_col='wiki_id',
	names=['wiki_id', 'fb_id', 'movie_name', 'movie_release', 'movie_revenue', 'movie_runtime', 'movie_languages', 'movie_countries', 'movie_genres']
)
# unwrap the mappings
df_movies.movie_languages = df_movies.movie_languages.map(lambda x: ",".join(list((json.loads(x).values()))))
df_movies.movie_countries = df_movies.movie_countries.map(lambda x: ",".join(list(json.loads(x).values())))
df_movies.movie_genres = df_movies.movie_genres.map(lambda x: ",".join(list(json.loads(x).values())))
df_movies['movie_release_year'] = df_movies.movie_release.apply(autoparse_year).astype('Int64')
df_movies['principal_genre']= df_movies.movie_genres.str.split(r",", expand=True)[0]
# TODO : convert dates to months also, where applicable

assert df_movies.fb_id.is_unique
assert df_movies.wiki_id.is_unique
assert df_movies.fb_id.is_unique

#### Correctness

In [5]:
df_movies[["movie_revenue", "movie_runtime", "movie_release_year"]].describe().apply(lambda s: s.apply('{:.5f}'.format))
# the fact the movie runtime can be zero is bit disturbing but we decided not to change it. The reason is that the film in question is of maximun length 1 minute.
# interesting fact is that there is dozen of movie that have duration less then 2 minutes, often old ones.

,movie_revenue,movie_runtime,movie_release_year
count,8401.00000,61291.00000,74839.00000
mean,47993625.22212,111.81918,1977.47653
std,112175292.92358,4360.07034,29.10154
min,10000.00000,0.00000,1010.00000
25%,2083193.00000,81.00000,1956.00000
50%,10639686.00000,93.00000,1985.00000
75%,40716963.00000,106.00000,2004.00000
max,2782275172.00000,1079281.00000,2016.00000


In [6]:
df_movies[df_movies.movie_release_year == 1010]

,wiki_id,fb_id,movie_name,movie_release,movie_revenue,movie_runtime,movie_languages,movie_countries,movie_genres,movie_release_year,principal_genre
62836,29666067,/m/0fphzrf,Hunting Season,1010-12-02,12160978.0,140.0,"Turkish Language,English Language",Turkey,"Crime Fiction,Mystery,Drama,Thriller",1010,Crime Fiction


In [7]:
df_movies.loc[df_movies.movie_release_year == 1010, 'movie_release'] = '2010-12-02'
df_movies.loc[df_movies.movie_release_year == 1010, 'movie_release_year'] = 2010

In [8]:
# checking the new distribution of movie_release_year
df_movies.movie_release_year.describe()

count    74839.000000
mean      1977.489892
std         28.886090
min       1888.000000
25%       1956.000000
50%       1985.000000
75%       2004.000000
max       2016.000000
Name: movie_release_year, dtype: float64

In [9]:
print(df_movies.movie_name.is_unique) # We want to see if no movie was reported twice
df_movies_dup = df_movies[df_movies.duplicated(subset=["movie_name", "movie_release_year", "movie_languages"])]
df_movies_dup = df_movies_dup.merge(df_movies_dup, on=["movie_name", "movie_release_year", "movie_languages"])
df_movies_dup[df_movies_dup.wiki_id_x != df_movies_dup.wiki_id_y].head() # display only head for readability purpose
# We conclude that those movies are quasi not differentiable because lots of information are missing about them and are therefore not relevant

False


,wiki_id_x,fb_id_x,movie_name,movie_release_x,movie_revenue_x,movie_runtime_x,movie_languages,movie_countries_x,movie_genres_x,movie_release_year,principal_genre_x,wiki_id_y,fb_id_y,movie_release_y,movie_revenue_y,movie_runtime_y,movie_countries_y,movie_genres_y,principal_genre_y
3,14597743,/m/03nm_pf,Vengeance,NaN,NaN,NaN,,,Action,<NA>,Action,27849952,/m/0cc8fj6,NaN,NaN,NaN,,"Crime Fiction,Thriller,Action,Drama,War film",Crime Fiction
4,27849952,/m/0cc8fj6,Vengeance,NaN,NaN,NaN,,,"Crime Fiction,Thriller,Action,Drama,War film",<NA>,Crime Fiction,14597743,/m/03nm_pf,NaN,NaN,NaN,,Action,Action
22,33545359,/m/0hgm4mx,A Sister to Assist 'Er,NaN,NaN,NaN,,,Comedy film,<NA>,Comedy film,33545449,/m/0hgrdzf,NaN,NaN,59.0,,Comedy film,Comedy film
23,33545359,/m/0hgm4mx,A Sister to Assist 'Er,NaN,NaN,NaN,,,Comedy film,<NA>,Comedy film,33545395,/m/0hgrll0,NaN,NaN,64.0,,Comedy film,Comedy film
24,33545359,/m/0hgm4mx,A Sister to Assist 'Er,NaN,NaN,NaN,,,Comedy film,<NA>,Comedy film,33545325,/m/0hgpm4_,NaN,NaN,NaN,,Comedy film,Comedy film


In [10]:
# We want to check the different languages, countries and genres and see if there were spelling errors or any other issue
# Checking by hand is enough since those list are quite limited, but we only print a subset here because of readability
unique_languages = np.unique(np.concatenate(df_movies.movie_languages.apply(lambda x: x.split(","))))
unique_countries = np.unique(np.concatenate(df_movies.movie_countries.apply(lambda x: x.split(","))))
unique_genres = np.unique(np.concatenate(df_movies.movie_genres.apply(lambda x: x.split(","))))
print(unique_languages[:20])
print(unique_countries[:20])
print(unique_genres[:20])

['' ' Adamawa Language' ' Ancient' ' Central Language' ' Hakka Language'
 ' Jinyu Language' ' North Language' ' Northeastern Language'
 ' Western Language' ' Yucatán Language' 'Aboriginal Malay languages'
 'Aceh Language' 'Afrikaans Language' 'Akan Language' 'Albanian language'
 'Algonquin Language' 'American English' 'American Sign Language'
 'Amharic Language' 'Ancient Greek']
['' 'Afghanistan' 'Albania' 'Algeria' 'Argentina' 'Armenia' 'Aruba'
 'Australia' 'Austria' 'Azerbaijan' 'Bahamas' 'Bahrain' 'Bangladesh'
 'Belgium' 'Bhutan' 'Bolivia' 'Bosnia and Herzegovina' 'Brazil' 'Bulgaria'
 'Burkina Faso']
['' 'Absurdism' 'Acid western' 'Action' 'Action Comedy' 'Action Thrillers'
 'Action/Adventure' 'Addiction Drama' 'Adult' 'Adventure'
 'Adventure Comedy' 'Airplanes and airports' 'Albino bias' 'Alien Film'
 'Alien invasion' 'Americana' 'Animal Picture' 'Animals'
 'Animated Musical' 'Animated cartoon']


#### Add rating data

In [11]:
tomato_scores = pd.read_csv("../data/raw/extra/movie_id2tomato_score.csv", skiprows=1, names=["fb_movie_id", "tomato_score"])
tomato_scores = tomato_scores.drop_duplicates(subset=["fb_movie_id"])
assert tomato_scores.fb_movie_id.is_unique

In [12]:
tomato_scores = tomato_scores[tomato_scores.tomato_score.str.match("^[-+]?\d+%$")]
tomato_scores.tomato_score = tomato_scores.tomato_score.str.strip("%").astype("float")

In [13]:
df_movies = pd.merge(df_movies, tomato_scores, how="left", left_on="fb_id", right_on="fb_movie_id")

In [14]:
with_tomato_scores = df_movies.tomato_score.count()
ratio = 100.0*(with_tomato_scores/len(df_movies))
print("There are %d rows with resolved tomato_scores. This is %.01f%% of rows"%(with_tomato_scores, ratio))

There are 14916 rows with resolved tomato_scores. This is 18.2% of rows


#### Add numerical freebase index

This is useful because igraph only takes numerical indices

In [15]:
# Commented out for now, because it's probably better to this when manipulating the igraph
# df_movies['movie_numfbid'] = df_movies.fb_id.factorize()[0]

#### Save

In [16]:
df_movies.to_pickle('../data/generated/preprocessed/movies.pkl')
df_movies

,wiki_id,fb_id,movie_name,movie_release,movie_revenue,movie_runtime,movie_languages,movie_countries,movie_genres,movie_release_year,principal_genre,fb_movie_id,tomato_score
0,975900,/m/03vyhn,Ghosts of Mars,2001-08-24,14010832.0,98.0,English Language,United States of America,"Thriller,Science Fiction,Horror,Adventure,Supe...",2001,Thriller,/m/03vyhn,22.0
1,3196793,/m/08yl5d,Getting Away with Murder: The JonBenét Ramsey ...,2000-02-16,NaN,95.0,English Language,United States of America,"Mystery,Biographical film,Drama,Crime Drama",2000,Mystery,NaN,NaN
2,28463795,/m/0crgdbh,Brun bitter,1988,NaN,83.0,Norwegian Language,Norway,"Crime Fiction,Drama",1988,Crime Fiction,NaN,NaN
3,9363483,/m/0285_cd,White Of The Eye,1987,NaN,110.0,English Language,United Kingdom,"Thriller,Erotic thriller,Psychological thriller",1987,Thriller,/m/0285_cd,50.0
4,261236,/m/01mrr1,A Woman in Flames,1983,NaN,106.0,German Language,Germany,Drama,1983,Drama,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...
81736,35228177,/m/0j7hxnt,Mermaids: The Body Found,2011-03-19,NaN,120.0,English Language,United States of America,Drama,2011,Drama,NaN,NaN
81737,34980460,/m/0g4pl34,Knuckle,2011-01-21,NaN,96.0,English Language,"Ireland,United Kingdom","Biographical film,Drama,Documentary",2011,Biographical film,/m/0g4pl34,93.0
81738,9971909,/m/02pygw1,Another Nice Mess,1972-09-22,NaN,66.0,English Language,United States of America,"Satire,Comedy",1972,Satire,NaN,NaN
81739,913762,/m/03pcrp,The Super Dimension Fortress Macross II: Lover...,1992-05-21,NaN,150.0,Japanese Language,Japan,"Science Fiction,Japanese Movies,Adventure,Anim...",1992,Science Fiction,NaN,NaN


### Character dataset

In [17]:
df_chars = pd.read_csv(
	'../data/raw/MovieSummaries/character.metadata.tsv', sep='\t',
	names=['wiki_id', 'fb_movie_id', 'release', 'character_name', 'actor_birth', 'actor_gender', 'actor_height', 'ethnicity', 'actor_name', 'actor_age', 'fb_char_id', 'useless_fb_char_id', 'fb_actor_id'],
)
df_chars = df_chars.drop("useless_fb_char_id", axis=1)
assert df_chars.index.is_unique
df_chars

,wiki_id,fb_movie_id,release,character_name,actor_birth,actor_gender,actor_height,ethnicity,actor_name,actor_age,fb_char_id,fb_actor_id
0,975900,/m/03vyhn,2001-08-24,Akooshay,1958-08-26,F,1.620,NaN,Wanda De Jesus,42.0,/m/0bgchxw,/m/03wcfv7
1,975900,/m/03vyhn,2001-08-24,Lieutenant Melanie Ballard,1974-08-15,F,1.780,/m/044038p,Natasha Henstridge,27.0,/m/0jys3m,/m/0346l4
2,975900,/m/03vyhn,2001-08-24,Desolation Williams,1969-06-15,M,1.727,/m/0x67,Ice Cube,32.0,/m/0jys3g,/m/01vw26l
3,975900,/m/03vyhn,2001-08-24,Sgt Jericho Butler,1967-09-12,M,1.750,NaN,Jason Statham,33.0,/m/02vchl6,/m/034hyc
4,975900,/m/03vyhn,2001-08-24,Bashira Kincaid,1977-09-25,F,1.650,NaN,Clea DuVall,23.0,/m/02vbb3r,/m/01y9xg
...,...,...,...,...,...,...,...,...,...,...,...,...
450664,913762,/m/03pcrp,1992-05-21,Elensh,1970-05,F,NaN,NaN,Dorothy Elias-Fahn,NaN,/m/0kr406c,/m/0b_vcv
450665,913762,/m/03pcrp,1992-05-21,Hibiki,1965-04-12,M,NaN,NaN,Jonathan Fahn,27.0,/m/0kr405_,/m/0bx7_j
450666,28308153,/m/0cp05t9,1957,NaN,1941-11-18,M,1.730,/m/02w7gg,David Hemmings,15.0,/m/0g8ngmc,/m/022g44
450667,28308153,/m/0cp05t9,1957,NaN,NaN,NaN,NaN,NaN,Roberta Paterson,NaN,/m/0g8ngmj,/m/0g8ngmm


### Correctness

In [18]:
# Check if actors attributes are the same across different movies
# Disclaimer: We are not expecting those assertion to be true, it is not impossible for any actor to change. 
# But in our case the information does not vary
for group, df_a in df_chars.groupby("fb_actor_id"):
    assert len(df_a.actor_name.unique()) == 1
    assert len(df_a.actor_birth.unique()) == 1
    assert len(df_a.actor_gender.unique()) == 1
    assert len(df_a.actor_height.unique()) == 1
# Check if dataset is coherent for movies
for group, df_m in df_chars.groupby("wiki_id"):
    assert len(df_m.fb_movie_id.unique()) == 1
    assert len(df_m.release.unique()) == 1

In [19]:
# we add the years only for praticality
df_chars['release_year'] = df_chars.release.apply(autoparse_year).astype('Int64')
df_chars['actor_birth_year'] = df_chars.actor_birth.apply(autoparse_year).astype('Int64')

In [20]:
# Let's look at the different distribution and then clean "non-normal" looking data
df_chars.drop("wiki_id", axis=1).describe().apply(lambda s: s.apply('{:.5f}'.format))

,actor_height,actor_age,release_year,actor_birth_year
count,154824.00000,292556.00000,440674.00000,344524.00000
mean,1.78889,37.78852,1984.48993,1943.10754
std,4.37994,20.58787,25.88952,34.11115
min,0.61000,-7896.00000,1010.00000,19.00000
25%,1.67640,28.00000,1969.00000,1923.00000
50%,1.75000,36.00000,1994.00000,1948.00000
75%,1.83000,47.00000,2005.00000,1966.00000
max,510.00000,103.00000,2016.00000,9900.00000


In [21]:
df_chars.loc[(df_chars.actor_birth_year >= 2023) | (df_chars.actor_birth_year <= 1750)]

,wiki_id,fb_movie_id,release,character_name,actor_birth,actor_gender,actor_height,ethnicity,actor_name,actor_age,fb_char_id,fb_actor_id,release_year,actor_birth_year
11550,21038272,/m/05b268g,1979,NaN,1498,F,NaN,NaN,Mirabai,NaN,/m/05cnn3x,/m/03h165,1979,1498
39427,5419425,/m/0dl0sj,2007-02-12,NaN,0019-11-28,NaN,NaN,NaN,Leland L. Jones,-12.0,/m/0gdl7t6,/m/0273sb0,2007,19
42283,2541685,/m/07lyw_,1993-06-16,NaN,0019-10-30,NaN,1.82,NaN,Dale Gibson,-26.0,/m/0gcjqyb,/m/0gcjqyg,1993,19
45080,2244529,/m/06yt3z,2003,NaN,0019-03-22,F,NaN,NaN,Julie Stevens,-16.0,/m/0bff0wc,/m/06p14x,2003,19
80339,997174,/m/03xvc7,2001-11-02,NaN,0019-11-28,NaN,NaN,NaN,Leland L. Jones,-18.0,/m/0647261,/m/0273sb0,2001,19
92871,20072112,/m/04n46f5,2008,NaN,1075-08-19,F,NaN,NaN,Holly Eglington,NaN,/m/0gc7_rh,/m/0gc64t0,2008,1075
107166,33952827,/m/0hnbcn2,2010,NaN,0852-03-10,M,NaN,NaN,Qian Liu,NaN,/m/0n64v09,/m/07gkxv,2010,852
137376,2236014,/m/06y5dn,1999,NaN,0195-02-07,F,1.67,NaN,Mónica Dionne,NaN,/m/09hyqcq,/m/02rvvq3,1999,195
155798,8060931,/m/026q2k7,NaN,NaN,0183,M,NaN,NaN,Lu Xun,NaN,/m/02vcff3,/m/01x879,<NA>,183
163428,5051761,/m/0d0l79,2006,NaN,0019-02-22,F,NaN,NaN,Dagney Kerr,-13.0,/m/0gc0gl6,/m/0fjlyh,2006,19


In [22]:
# found most of the values on wikipedia, the rest it seems to be that actor_name was replaced by the character_name, we therefore have to drop them
df_chars.loc[df_chars.actor_birth_year == 1075, 'actor_birth'] = '1975-11-28'
df_chars.loc[df_chars.actor_birth_year == 1075, 'actor_birth_year'] = 1975
df_chars.loc[df_chars.actor_name == "Leland L. Jones", 'actor_birth'] = '1963-11-28'
df_chars.loc[df_chars.actor_name == "Leland L. Jones", 'actor_birth_year'] = 1963
df_chars.loc[df_chars.actor_name == "Mónica Dionne", 'actor_birth'] = '1967-02-07'
df_chars.loc[df_chars.actor_name == "Mónica Dionne", 'actor_birth_year'] = 1967
df_chars.loc[df_chars.actor_name == "Dwayne L. Barnes", 'actor_birth'] = '1970-11-26'
df_chars.loc[df_chars.actor_name == "Dwayne L. Barnes", 'actor_birth_year'] = 1970
df_chars.loc[df_chars.actor_name == "Dagney Kerr", 'actor_birth'] = '1972-02-22'
df_chars.loc[df_chars.actor_name == "Dagney Kerr", 'actor_birth_year'] = 1972
df_chars.loc[df_chars.actor_birth_year == 2896, 'actor_birth'] = '1896-04-26'
df_chars.loc[df_chars.actor_birth_year == 2896, 'actor_birth_year'] = 1896
df_chars.loc[df_chars.actor_birth_year == 9900, 'actor_birth'] = '1900-09-07'
df_chars.loc[df_chars.actor_birth_year == 9900, 'actor_birth_year'] = 1900
df_chars.loc[df_chars.actor_birth_year == 2936, 'actor_birth'] = '1936-12-24'
df_chars.loc[df_chars.actor_birth_year == 2936, 'actor_birth_year'] = 1936
df_chars.loc[df_chars.actor_birth_year == 2050, 'actor_birth'] = '1971-12-06'
df_chars.loc[df_chars.actor_birth_year == 2050, 'actor_birth_year'] = 1971
char_names_instead = ["Mirabai", "Qian Liu", "Juan Diego Cuauhtlatoatzin", "Lu Xun"]
df_chars = df_chars[df_chars.actor_name.isin(char_names_instead) == False]
not_found = ["Dale Gibson", "Julie Stevens"]
df_chars.loc[df_chars.actor_name.isin(not_found), ["actor_age", "actor_birth", "actor_birth_year"]] = np.NAN

In [23]:
df_chars[df_chars.release_year == 1010]

,wiki_id,fb_movie_id,release,character_name,actor_birth,actor_gender,actor_height,ethnicity,actor_name,actor_age,fb_char_id,fb_actor_id,release_year,actor_birth_year
67624,29666067,/m/0fphzrf,1010-12-02,Kamuran,1983,M,NaN,NaN,Bartu Küçükçaglayan,NaN,/m/0n4fxh5,/m/0df0qkt,1010,1983
67625,29666067,/m/0fphzrf,1010-12-02,Ferman,1941-12-26,M,NaN,NaN,Şener Şen,NaN,/m/0gb33kh,/m/02z247j,1010,1941
67626,29666067,/m/0fphzrf,1010-12-02,Idris,1973-04-23,M,NaN,NaN,Cem Yılmaz,NaN,/m/0gb33kn,/m/0fz9xn,1010,1973
67627,29666067,/m/0fphzrf,1010-12-02,Hasan,1978-12-13,M,NaN,NaN,Okan Yalabık,NaN,/m/0ggd8h0,/m/0cmb83k,1010,1978
67628,29666067,/m/0fphzrf,1010-12-02,Battal,1945-07-16,M,NaN,NaN,Çetin Tekindor,NaN,/m/0h9dtmf,/m/0ch5v5f,1010,1945
67629,29666067,/m/0fphzrf,1010-12-02,Asiye,1985-07-06,F,NaN,NaN,Melisa Sözen,NaN,/m/0k3m4yc,/m/0ch3v_t,1010,1985
67630,29666067,/m/0fphzrf,1010-12-02,Asit Omer,NaN,M,NaN,NaN,Riza Kocaoglu,NaN,/m/0n4fssj,/m/0n4fssv,1010,<NA>
67631,29666067,/m/0fphzrf,1010-12-02,Hatun,NaN,F,NaN,NaN,Nergis Çorakçi,NaN,/m/0n4ftj1,/m/0n4ftjc,1010,<NA>
67632,29666067,/m/0fphzrf,1010-12-02,Müslüm,1963,M,NaN,NaN,Mustafa Avkiran,NaN,/m/0n4fv9p,/m/0gc9xb7,1010,1963
67633,29666067,/m/0fphzrf,1010-12-02,Murat Önes,NaN,M,NaN,NaN,Mahir Ipek,NaN,/m/0n4fvtw,/m/0n4fvv5,1010,<NA>


In [24]:
df_chars.loc[df_chars.release_year == 1010, 'release'] = '2010-12-02'
df_chars.loc[df_chars.release_year == 1010, 'release_year'] = 2010

In [25]:
df_chars[df_chars.release_year < df_chars.actor_birth_year]
# By looking at the movies in question, it is sometimes the release_year and sometimes the actor_birth_year that is faulty

,wiki_id,fb_movie_id,release,character_name,actor_birth,actor_gender,actor_height,ethnicity,actor_name,actor_age,fb_char_id,fb_actor_id,release_year,actor_birth_year
767,15007384,/m/03h4h5p,1934-05-02,NaN,1963-11-07,NaN,NaN,NaN,Franck Dubosc,-29.0,/m/0bwb9k3,/m/01wlly9,1934,1963
2286,2553539,/m/07mlsy,1918-04-14,NaN,1931-03-25,M,NaN,NaN,Tom Wilson,-12.0,/m/0jsqm1,/m/0382zh,1918,1931
3892,5917218,/m/0fdf3y,1965,NaN,1983-03-03,F,1.75,NaN,Kim Smith,-18.0,/m/04mb5kf,/m/03pjkv,1965,1983
6666,73930,/m/0jsjm,1924,Robert,1972-11-07,M,1.80,NaN,Jason London,-48.0,/m/01xrptw,/m/04kdmg,1924,1972
7188,21242801,/m/05bzjwj,1955-08-07,NaN,1973-08-01,M,1.79,/m/03ttfc,Eduardo Noriega,-17.0,/m/0cg2f0f,/m/0bkkw0,1955,1973
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
446581,25868401,/m/09v6ykm,1955,NaN,1967-05-31,F,1.67,/m/03ts0c,Sandrine Bonnaire,-12.0,/m/04j05tr,/m/07wn_5,1955,1967
446583,8595532,/m/02799cv,1944-02-23,NaN,1947-05-28,M,NaN,NaN,Kevin O'Shea,-3.0,/m/02vcqnq,/m/02q384f,1944,1947
446816,1276915,/m/04pg6m,1941-06-20,NaN,1957-04-19,M,NaN,NaN,Tony Martin,-15.0,/m/0cg164j,/m/07q_fg,1941,1957
447210,23382222,/m/06w1hlj,1932-08-09,NaN,1942-02-08,M,NaN,/m/041rx,Robert Klein,-9.0,/m/0cg08dz,/m/02pb53,1932,1942


In [26]:
# But the actor were still present in the films, so instead of dropping those faulty rows,
# we keep them since membership is still interesting to us, but we put Na for the incertain columns
df_chars.loc[df_chars.release_year < df_chars.actor_birth_year, ["release", "release_year", "actor_age", "actor_birth", "actor_birth_year"]] = np.NAN

In [27]:
df_chars.loc[df_chars.actor_age < 0]

,wiki_id,fb_movie_id,release,character_name,actor_birth,actor_gender,actor_height,ethnicity,actor_name,actor_age,fb_char_id,fb_actor_id,release_year,actor_birth_year
39427,5419425,/m/0dl0sj,2007-02-12,NaN,1963-11-28,NaN,NaN,NaN,Leland L. Jones,-12.0,/m/0gdl7t6,/m/0273sb0,2007,1963
80339,997174,/m/03xvc7,2001-11-02,NaN,1963-11-28,NaN,NaN,NaN,Leland L. Jones,-18.0,/m/0647261,/m/0273sb0,2001,1963
163428,5051761,/m/0d0l79,2006,NaN,1972-02-22,F,NaN,NaN,Dagney Kerr,-13.0,/m/0gc0gl6,/m/0fjlyh,2006,1972
246642,5313889,/m/0df13_,2007,NaN,1970-11-26,M,1.8,NaN,Dwayne L. Barnes,-12.0,/m/0gckdwz,/m/0gc91mg,2007,1970
290660,6322722,/m/0g0_4m,2006-09-08,NaN,1963-11-28,NaN,NaN,NaN,Leland L. Jones,-13.0,/m/0gdnptr,/m/0273sb0,2006,1963
300065,22643470,/m/05zqsh4,1926-10-29,NaN,1896-04-26,M,NaN,NaN,Matthieu van Eysden,-969.0,/m/0n1fz80,/m/0jt60hb,1926,1896
322333,10240061,/m/02q674n,2004,NaN,1900-09-07,F,NaN,NaN,Kimberly Jones,-7896.0,/m/02vcpws,/m/0cc1lj,2004,1900
325421,7065434,/m/0h2m37,2001,NaN,1963-11-28,NaN,NaN,NaN,Leland L. Jones,-18.0,/m/0gdk7gn,/m/0273sb0,2001,1963
345994,34654608,/m/0j26ybm,2012-02-04,NaN,1971-12-06,M,NaN,NaN,Naozumi Takahashi,-37.0,/m/0n9z5lz,/m/044pxh,2012,1971
350343,19152242,/m/04lf47x,2009-07-14,Reverend Dawkins,1963-11-28,NaN,NaN,NaN,Leland L. Jones,-10.0,/m/0h6cd22,/m/0273sb0,2009,1963


In [28]:
# We recompute the age of the actor for the faulty value
df_chars.loc[df_chars.actor_age < 0, "actor_age"] = df_chars.loc[df_chars.actor_age < 0].release_year - df_chars.loc[df_chars.actor_age < 0].actor_birth_year

In [29]:
df_chars[df_chars.actor_age > 90] 
# there quite some old people, it will take some time to verify them one by one, so we leave them be !

,wiki_id,fb_movie_id,release,character_name,actor_birth,actor_gender,actor_height,ethnicity,actor_name,actor_age,fb_char_id,fb_actor_id,release_year,actor_birth_year
5646,142443,/m/011yl_,1996-01-21,Cecil Parkes,1904-04-14,M,1.80,NaN,John Gielgud,91.0,/m/0bnr0sv,/m/016ggh,1996,1904
11235,24551438,/m/080jym_,2009-09-04,Emil Pikler,1915-12-07,M,1.78,/m/041rx,Eli Wallach,93.0,/m/08c_xs1,/m/02vg0,2009,1915
14061,7494411,/m/0263fpm,2005-09-10,NaN,1906-07-08,M,NaN,NaN,Philip Johnson,99.0,/m/0h1k3wt,/m/0d__g,2005,1906
14738,19442420,/m/04n6h0f,2006,NaN,1908-11-04,M,NaN,/m/01p7s6,Joseph Rotblat,97.0,/m/0h1l8zn,/m/01ttdp,2006,1908
23510,22237851,/m/05q89qb,2008,NaN,1911-12-25,F,NaN,NaN,Louise Bourgeois,96.0,/m/0h1kqct,/m/02vnb6,2008,1911
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
436393,5451497,/m/0dmh22,2008-02-01,Milas,1917-01-24,M,1.75,NaN,Ernest Borgnine,91.0,/m/04m9sbr,/m/015d3h,2008,1917
439566,31562257,/m/0glp976,2010-11-05,NaN,1919-03-05,M,NaN,/m/0dryh9k,M. N. Nambiar,91.0,/m/0h73h8m,/m/0cd6_9,2010,1919
443215,31129643,/m/0cryb4w,2000,NaN,1908-09-01,M,NaN,/m/0dryh9k,K.N. Singh,91.0,/m/0j4mrqm,/m/03bxsk4,2000,1908
443838,25398933,/m/09vtyqn,2002-10-12,NaN,1909-02-04,M,NaN,NaN,Charles Simon,93.0,/m/0gm1zgx,/m/0gm1zgz,2002,1909


In [30]:
df_chars[df_chars.actor_height > 200]

,wiki_id,fb_movie_id,release,character_name,actor_birth,actor_gender,actor_height,ethnicity,actor_name,actor_age,fb_char_id,fb_actor_id,release_year,actor_birth_year
49659,30865068,/m/02ml14,1998-10-15,NaN,1972-12-06,M,510.0,NaN,Vince Corazza,25.0,/m/0n9ld0y,/m/08hqh0,1998,1972
104140,17999628,/m/047n_2q,1998,NaN,1972-12-06,M,510.0,NaN,Vince Corazza,25.0,/m/0cg31bd,/m/08hqh0,1998,1972
205679,1373475,/m/04xmx8,2003-08-15,Jackal Johnson,1972-12-06,M,510.0,NaN,Vince Corazza,30.0,/m/02vbm0k,/m/08hqh0,2003,1972
301653,4667507,/m/0cg9n6,2001-11-29,NaN,1972-12-06,M,510.0,NaN,Vince Corazza,28.0,/m/0cg0zwp,/m/08hqh0,2001,1972
316621,3915041,/m/0b6lyp,2002,NaN,1972-12-06,M,510.0,NaN,Vince Corazza,29.0,/m/02vc5p5,/m/08hqh0,2002,1972
341890,29290087,/m/0b3w9mr,1997-04-07,NaN,1972-12-06,M,510.0,NaN,Vince Corazza,24.0,/m/0gvwy8x,/m/08hqh0,1997,1972
369731,33945838,/m/0hn8wrq,2012-09-29,Max Loflin,NaN,M,510.0,NaN,Benedict Smith,NaN,/m/0n3kptl,/m/0n3kptx,2012,<NA>
380739,4738411,/m/0ckr7s,1993-12-05,Darien - Tuxedo Mask,1972-12-06,M,510.0,NaN,Vince Corazza,20.0,/m/0bmz820,/m/08hqh0,1993,1972
393292,1244822,/m/04lq2l,2003-01-23,NaN,1972-12-06,M,510.0,NaN,Vince Corazza,30.0,/m/0bfm1b6,/m/08hqh0,2003,1972
406151,4578386,/m/0c9sj8,2004-04-06,NaN,1972-12-06,M,510.0,NaN,Vince Corazza,31.0,/m/03lj_70,/m/08hqh0,2004,1972


In [31]:
# Vince Corazza has a height of 178 cm
# Benedict Smith has a heigh of 180 cm
df_chars.loc[df_chars.actor_name == "Vince Corazza", "actor_height"] = 1.78
df_chars.loc[df_chars.actor_name == "Benedict Smith", "actor_height"] = 1.80

In [32]:
df_chars[df_chars.actor_height > 100]

,wiki_id,fb_movie_id,release,character_name,actor_birth,actor_gender,actor_height,ethnicity,actor_name,actor_age,fb_char_id,fb_actor_id,release_year,actor_birth_year
21619,1291587,/m/04qk12,2004-09-01,young Georgie,1986-03-13,M,180.0,/m/0dc58y8,Zohren Weiss,18.0,/m/0k276dx,/m/0bggwvl,2004,1986
59719,12206655,/m/02vvw24,NaN,Dominic,1986-03-13,M,180.0,/m/0dc58y8,Zohren Weiss,NaN,/m/0k276dq,/m/0bggwvl,<NA>,1986
219383,179326,/m/018gq3,1996-11-18,Herbert,1986-03-13,M,180.0,/m/0dc58y8,Zohren Weiss,10.0,/m/0bggwv8,/m/0bggwvl,1996,1986
360967,14413223,/m/03d2rnl,2012-04-20,Young Punk Guy,1986-03-13,M,180.0,/m/0dc58y8,Zohren Weiss,26.0,/m/0m4q87d,/m/0bggwvl,2012,1986


In [33]:
# put height on the right scale
df_chars.loc[df_chars.actor_name == "Zohren Weiss", "actor_height"] = 1.80

In [34]:
# Let's look at the new distributions
df_chars.drop("wiki_id", axis=1).describe().apply(lambda s: s.apply('{:.5f}'.format))

,actor_height,actor_age,release_year,actor_birth_year
count,154824.00000,292184.00000,440266.00000,344113.00000
mean,1.74818,37.89151,1984.55471,1943.13323
std,0.10624,14.08598,25.24149,28.95474
min,0.61000,0.00000,1888.00000,1796.00000
25%,1.67640,28.00000,1969.00000,1923.00000
50%,1.75000,36.00000,1994.00000,1948.00000
75%,1.83000,47.00000,2005.00000,1966.00000
max,2.35600,104.00000,2016.00000,2008.00000


In [35]:
# Check if the release, birth and age are coherent
df_chars_year_relevant = df_chars.dropna(subset=["release_year", "actor_birth_year", "actor_age"])
assert all((df_chars_year_relevant.release_year - df_chars_year_relevant.actor_birth_year - df_chars_year_relevant.actor_age).abs() <= 1) # we leave a one year leeway 

In [36]:
# Check coherence between movies and actors dataset
df_chars_m = df_chars.drop_duplicates(subset=["wiki_id"], keep="first")[["wiki_id", "fb_movie_id", "release", "release_year"]]
df_movies_c = df_movies[["wiki_id", "fb_id", "movie_name", "movie_release", "movie_release_year"]]
df_movies_chars = df_movies_c.merge(df_chars_m, on="wiki_id")
del df_movies_c, df_chars_m
assert all(df_movies_chars.fb_id == df_movies_chars.fb_movie_id)
df_movies_chars.dropna(subset=["release_year", "movie_release_year"], inplace=True)
assert all(df_movies_chars.release == df_movies_chars.movie_release)
assert all(df_movies_chars.release_year == df_movies_chars.movie_release_year)

### Actor dataset

In [37]:
df_actors = df_chars[["fb_actor_id", "actor_name", "actor_birth", "ethnicity", "actor_height", "actor_gender"]]
df_actors = df_actors.drop_duplicates(subset="fb_actor_id")
df_chars = df_chars.drop(["actor_name", "actor_birth", "ethnicity", "actor_height", "actor_gender"], axis=1)
df_chars.to_pickle('../data/generated/preprocessed/roles.pkl')

In [38]:
df_actor2nationality_id = pd.read_csv("../data/raw/extra/actor_id2nationality_id.csv", skiprows=1, names=["fb_actor_id", "nationality_id"])
df_nationality_id2nationality = pd.read_csv("../data/raw/extra/nationality_id2nationality.csv", skiprows=1, names=["nationality_id", "actor_nationality"])
df_nationality = pd.merge(df_actor2nationality_id, df_nationality_id2nationality, on="nationality_id")
df_nationality = df_nationality.drop("nationality_id", axis=1)

In [39]:
df_nationality = df_nationality.drop_duplicates(subset=["fb_actor_id"])

In [40]:
df_actors = pd.merge(df_actors, df_nationality, on="fb_actor_id", how="left")


In [41]:
with_nationnalities = df_actors.actor_nationality.count()
ratio = 100.0*(with_nationnalities/len(df_actors))
print("There are %d actors with resolved nationalities. This is %.01f%% of actors"%(with_nationnalities, ratio))

There are 55606 actors with resolved nationalities. This is 41.0% of actors


In [42]:
# Ethnic groups mapping (queried from wikidata)
ethnic_groups = pd.read_csv('../data/raw/extra/ethnic_groups.csv', index_col="freebaseID")
found_ethnicities = pd.merge(df_actors, ethnic_groups, how="left", left_on="ethnicity", right_on="freebaseID")
now_count = found_ethnicities.ethnicity.nunique()
previous_count = df_actors.ethnicity.nunique()
ratio = 100.0*(df_actors.ethnicity.count()/len(df_actors))
print("There were %d distinct ethnicity IDs, and we can resolve %d of them. This is %.1f%% of actors' ethnicities"% (previous_count, now_count, ratio))

dictionnary = {}
for fbID, name in ethnic_groups.name.items():
    dictionnary[fbID] = name

df_actors.ethnicity = df_actors.ethnicity.map(dictionnary)

There were 479 distinct ethnicity IDs, and we can resolve 479 of them. This is 6.0% of actors' ethnicities


#### Add numerical freebase index

In [43]:
# Commented out for now, because it's probably better to this when manipulating the igraph
# df_actors['actor_numfbid'] = df_actors.fb_actor_id.factorize()[0]

#### Save

In [44]:
df_actors.to_pickle('../data/generated/preprocessed/actors.pkl')
df_actors

,fb_actor_id,actor_name,actor_birth,ethnicity,actor_height,actor_gender,actor_nationality
0,/m/03wcfv7,Wanda De Jesus,1958-08-26,NaN,1.620,F,United States of America
1,/m/0346l4,Natasha Henstridge,1974-08-15,NaN,1.780,F,Canada
2,/m/01vw26l,Ice Cube,1969-06-15,NaN,1.727,M,United States of America
3,/m/034hyc,Jason Statham,1967-09-12,NaN,1.750,M,United Kingdom
4,/m/01y9xg,Clea DuVall,1977-09-25,NaN,1.650,F,United States of America
...,...,...,...,...,...,...,...
135752,/m/0kr409z,Violet Bronte,NaN,NaN,NaN,F,NaN
135753,/m/0gn4bz,Sonny Byrkett,1954,NaN,NaN,M,NaN
135754,/m/0gn4nd,Susan Byrkett,1958,NaN,NaN,F,NaN
135755,/m/0kr407h,Hal Cleaveland,NaN,NaN,NaN,M,NaN


### Characters

#### Update Character table with preprocessed Actor information (for compatibility)

In [45]:
df_chars = pd.merge(df_chars, df_actors, on="fb_actor_id")
df_chars

,wiki_id,fb_movie_id,release,character_name,actor_age,fb_char_id,fb_actor_id,release_year,actor_birth_year,actor_name,actor_birth,ethnicity,actor_height,actor_gender,actor_nationality
0,975900,/m/03vyhn,2001-08-24,Akooshay,42.0,/m/0bgchxw,/m/03wcfv7,2001,1958,Wanda De Jesus,1958-08-26,NaN,1.62,F,United States of America
1,24767455,/m/0809s7n,2009-10-16,Captain Diaz,51.0,/m/08d9z0b,/m/03wcfv7,2009,1958,Wanda De Jesus,1958-08-26,NaN,1.62,F,United States of America
2,5337169,/m/0dg7mt,2001,NaN,42.0,/m/04htxdc,/m/03wcfv7,2001,1958,Wanda De Jesus,1958-08-26,NaN,1.62,F,United States of America
3,1330220,/m/04t8wq,1990-06-22,NaN,31.0,/m/0cg7kl2,/m/03wcfv7,1990,1958,Wanda De Jesus,1958-08-26,NaN,1.62,F,United States of America
4,14686017,/m/03gt21w,1990-01-12,NaN,31.0,/m/09j0r8t,/m/03wcfv7,1990,1958,Wanda De Jesus,1958-08-26,NaN,1.62,F,United States of America
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
450658,913762,/m/03pcrp,1992-05-21,Additional Voices,38.0,/m/0kr405f,/m/0gn4bz,1992,1954,Sonny Byrkett,1954,NaN,NaN,M,NaN
450659,913762,/m/03pcrp,1992-05-21,UN Spacy Commander,38.0,/m/0kr407w,/m/0gn4bz,1992,1954,Sonny Byrkett,1954,NaN,NaN,M,NaN
450660,913762,/m/03pcrp,1992-05-21,Silvie Gena,34.0,/m/0kr40b9,/m/0gn4nd,1992,1958,Susan Byrkett,1958,NaN,NaN,F,NaN
450661,913762,/m/03pcrp,1992-05-21,Exegran,NaN,/m/0kr407c,/m/0kr407h,1992,<NA>,Hal Cleaveland,NaN,NaN,NaN,M,NaN


#### Save

In [46]:
df_chars.to_pickle('../data/generated/preprocessed/characters.pkl')

In [47]:
with_ethnicities = df_chars.ethnicity.count()
ratio = 100.0 * with_ethnicities/len(df_chars)
print("%d rows, or %.1f%% of character ethnicities are resolved."%(with_ethnicities, ratio))

48650 rows, or 10.8% of character ethnicities are resolved.


### TV tropes

In [48]:
df_tropes = pd.read_csv(
	'../data/raw/MovieSummaries/tvtropes.clusters.txt',
	sep='\t',
	names=['trope', 'char_movie_id']
)
df_tropes['char_name'] = df_tropes.char_movie_id.map(lambda x: json.loads(x)['char'])
df_tropes['movie_name'] = df_tropes.char_movie_id.map(lambda x: json.loads(x)['movie'])
df_tropes['actor_name'] = df_tropes.char_movie_id.map(lambda x: json.loads(x)['actor'])
df_tropes['fb_id'] = df_tropes.char_movie_id.map(lambda x: json.loads(x)['id'])
df_tropes.drop(columns='char_movie_id', inplace=True)
df_tropes

,trope,char_name,movie_name,actor_name,fb_id
0,absent_minded_professor,Professor Philip Brainard,Flubber,Robin Williams,/m/0jy9q0
1,absent_minded_professor,Professor Keenbean,Richie Rich,Michael McShane,/m/02vchl3
2,absent_minded_professor,Dr. Reinhardt Lane,The Shadow,Ian McKellen,/m/0k6fkc
3,absent_minded_professor,Dr. Harold Medford,Them!,Edmund Gwenn,/m/0k6_br
4,absent_minded_professor,Daniel Jackson,Stargate,James Spader,/m/0k3rhh
...,...,...,...,...,...
496,young_gun,Morgan Earp,Tombstone,Bill Paxton,/m/0k776f
497,young_gun,Colorado Ryan,Rio Bravo,Ricky Nelson,/m/0k2kqg
498,young_gun,Tom Sawyer,The League of Extraordinary Gentlemen,Shane West,/m/0k5nsh
499,young_gun,William H. 'Billy the Kid' Bonney,Young Guns II,Emilio Estevez,/m/03lrjk0


### Correctness

In [49]:
# Check coherence between datasets
df_pre_1 = df_chars.drop_duplicates(subset=["wiki_id"], keep="first")[["actor_name", "fb_actor_id"]]
df_pre_2 = df_tropes[["actor_name", "fb_id"]].rename(columns={"fb_id": "fb_actor_id"})
df_pre = df_pre_1.merge(df_pre_2, on="fb_actor_id")
assert all(df_pre.actor_name_x == df_pre.actor_name_y)

In [50]:
df_tropes.to_pickle('../data/generated/preprocessed/tropes.pkl')

### Character clusters

In [51]:
df_charclusters = pd.read_csv(
	'../data/raw/MovieSummaries/name.clusters.txt',
	sep='\t',
	names=['char_name', 'fb_id']
)
df_charclusters

,char_name,fb_id
0,Stuart Little,/m/0k3w9c
1,Stuart Little,/m/0k3wcx
2,Stuart Little,/m/0k3wbn
3,John Doe,/m/0jyg35
4,John Doe,/m/0k2_zn
...,...,...
2661,John Rolfe,/m/0k5_ql
2662,John Rolfe,/m/02vd6vs
2663,Elizabeth Swann,/m/0k1xvz
2664,Elizabeth Swann,/m/0k1x_d


In [52]:
df_charclusters.to_pickle('../data/generated/preprocessed/character_clusters.pkl')